# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [2]:
import pandas as pd
import numpy as np


## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [2]:
# Following are shell commands to download the data
! mkdir -p data
!cd data ; curl -O  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv ; cd ..

The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.


In [3]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [4]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')



Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

In [5]:
pop.head(2)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0


In [6]:
abbrevs.head(2)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK


In [9]:
result = pop.append(abbrevs, ignore_index=False, sort=False)
result.head(2)

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012.0,1117489.0,NaN,NaN
1,AL,total,2012.0,4817528.0,NaN,NaN


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

## Look for any rows with null values in the new dataframe

In [23]:
result.isnull().any()

state/region    True
ages            True
year            True
population      True
state           True
abbreviation    True
dtype: bool

## Print out the rows in the merged dataframe that has null values for population.

It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [21]:
result.loc[result['state'].isnull(), 'state/region'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR', 'USA'],
      dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [25]:
result.loc[result['state/region'] == 'PR', 'state'] = 'Puerto Rico'
result.loc[result['state/region'] == 'USA', 'state'] = 'United States'
result.isnull().any()

state/region    True
ages            True
year            True
population      True
state           True
abbreviation    True
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [26]:
final = pd.merge(result, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012.0,1117489.0,NaN,NaN,NaN
1,AL,total,2012.0,4817528.0,NaN,NaN,NaN
2,AL,under18,2010.0,1130966.0,NaN,NaN,NaN
3,AL,total,2010.0,4785570.0,NaN,NaN,NaN
4,AL,under18,2011.0,1125763.0,NaN,NaN,NaN


## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

## Select the portion of the data corresponding with the year 2000, and the total population.

## Re-index the data on state and find population density and display by density in ascending order

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!